<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Ejercicio_PySpark_RutasGPS_Transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚛 Análisis de rutas GPS en el sector transporte usando PySpark
Este ejercicio simula el procesamiento de registros GPS para una empresa del sector transporte. Se analizará la distancia recorrida y la eficiencia de los trayectos usando PySpark.

## 🎯 Objetivo
Simular datos de rutas de vehículos y calcular distancias totales por vehículo, trayectos promedio y eficiencia de rutas usando procesamiento distribuido.

In [1]:
# Paso 1: Inicializar sesión de Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RutasGPS_Transporte").getOrCreate()

In [2]:
# Paso 2: Crear datos simulados de rutas GPS (vehículo, fecha, distancia_km, duración_minutos)
data = [
    ("V001", "2024-01-01", 120.5, 150),
    ("V001", "2024-01-02", 130.0, 160),
    ("V002", "2024-01-01", 95.0, 120),
    ("V002", "2024-01-02", 100.0, 130),
    ("V003", "2024-01-01", 80.0, 110),
    ("V003", "2024-01-02", 90.0, 115),
]
columnas = ["vehiculo", "fecha", "distancia_km", "duracion_min"]
df = spark.createDataFrame(data, columnas)
df.show()

+--------+----------+------------+------------+
|vehiculo|     fecha|distancia_km|duracion_min|
+--------+----------+------------+------------+
|    V001|2024-01-01|       120.5|         150|
|    V001|2024-01-02|       130.0|         160|
|    V002|2024-01-01|        95.0|         120|
|    V002|2024-01-02|       100.0|         130|
|    V003|2024-01-01|        80.0|         110|
|    V003|2024-01-02|        90.0|         115|
+--------+----------+------------+------------+



In [3]:
# Paso 3: Calcular distancia total recorrida por vehículo
from pyspark.sql.functions import sum
df.groupBy("vehiculo").agg(sum("distancia_km").alias("total_km")).show()

+--------+--------+
|vehiculo|total_km|
+--------+--------+
|    V001|   250.5|
|    V002|   195.0|
|    V003|   170.0|
+--------+--------+



In [7]:
# Paso 4: Calcular tiempo promedio por trayecto y velocidad promedio
from pyspark.sql.functions import avg, round
df = df.withColumn("velocidad_promedio", round(df["distancia_km"] / (df["duracion_min"] / 60), 2))
df.select("vehiculo", "fecha", "distancia_km", "duracion_min", "velocidad_promedio").show()

+--------+----------+------------+------------+------------------+
|vehiculo|     fecha|distancia_km|duracion_min|velocidad_promedio|
+--------+----------+------------+------------+------------------+
|    V001|2024-01-01|       120.5|         150|              48.2|
|    V001|2024-01-02|       130.0|         160|             48.75|
|    V002|2024-01-01|        95.0|         120|              47.5|
|    V002|2024-01-02|       100.0|         130|             46.15|
|    V003|2024-01-01|        80.0|         110|             43.64|
|    V003|2024-01-02|        90.0|         115|             46.96|
+--------+----------+------------+------------+------------------+



In [9]:
# Paso 5: Comparar eficiencia entre vehículos
df.groupBy("vehiculo") \
  .agg(avg("velocidad_promedio").alias("vel_promedio_general")) \
  .orderBy("vel_promedio_general", ascending=False) \
  .show()

+--------+--------------------+
|vehiculo|vel_promedio_general|
+--------+--------------------+
|    V001|              48.475|
|    V002|              46.825|
|    V003|                45.3|
+--------+--------------------+



## ✅ Conclusión
- Este ejercicio demuestra cómo Spark puede procesar rápidamente grandes volúmenes de registros GPS.
- En contextos reales, esto se escala a millones de registros diarios.
- Usando clústeres Spark con 10 nodos, el tiempo de análisis puede reducirse de 6 horas a menos de 30 minutos.
- Estos análisis permiten optimizar rutas, reducir costos operativos y mejorar tiempos de entrega en el sector transporte.